# 한글깨짐

In [ ]:
# 단계 1: 폰트 설치
import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()

In [ ]:
# 단계 2: 런타임 재시작
# import os
# os.kill(os.getpid(), 9)

In [ ]:
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False
	
# 한글 폰트 설정
path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
font_name = fm.FontProperties(fname=path, size=18).get_name()
plt.rc('font', family=font_name)
fm._rebuild()

# 1. 환경설정

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd '/content/gdrive/MyDrive/class7/00_온반오반 팀/핵심역량프로젝트/1. 지성_코딩자료/최적화모델찾기'
!ls

/content/gdrive/.shortcut-targets-by-id/1N1s8vhguqw4WIGdcNXcBhWiF00DpHk-W/온반오반 팀/핵심역량프로젝트/1. 지성_코딩자료/최적화모델찾기
catboost_info	     어진.ipynb		   OWID_KOREA_COVID데이터_all.csv
center_to_local.csv  재은.ipynb		   select_jump_up.csv
data.csv	     은석.ipynb
지성.ipynb	     분류적용_last_df.csv


In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df['측정일'] = df['년'].astype(int).astype(str) + df['월'].astype(int).astype(str).map(lambda x: x if len(x) == 2 else '0'+x) + df['일'].astype(int).astype(str).map(lambda x: x if len(x) == 2 else '0'+x)
df['측정일'] = df['측정일'].astype(int)

In [ ]:
df.tail()

,년,월,일,요일,회차,나이,성별,신장,체중,체지방률,이완기혈압,수축기혈압,BMI,유연성(cm),일리노이,체공시간,협응력,심폐지구력,근력,학교폐쇄,모임제한,대중교통폐쇄,시설제한,측정일
303108,2022.00,5.00,30.00,Monday,2.00,18.00,0.00,177.70,70.90,20.20,77.00,142.00,22.50,6.30,17.46,0.54,48.30,60.98,55.10,0.00,0.00,0.00,8.57,20220530
303109,2022.00,5.00,30.00,Monday,1.00,18.00,1.00,165.50,60.80,29.80,71.00,109.00,22.20,31.50,19.21,0.51,75.29,23.00,46.70,0.00,0.00,0.00,8.57,20220530
303110,2022.00,5.00,30.00,Monday,2.00,16.00,1.00,167.60,61.50,22.70,62.00,139.00,21.90,13.70,19.86,0.50,55.48,32.00,63.60,0.00,0.00,0.00,8.57,20220530
303111,2022.00,5.00,30.00,Monday,1.00,16.00,1.00,152.80,49.70,30.60,72.00,123.00,21.30,15.30,21.14,0.45,43.99,31.00,50.70,0.00,0.00,0.00,8.57,20220530
303112,2022.00,5.00,30.00,Monday,1.00,15.00,0.00,158.60,50.20,14.00,77.00,121.00,20.00,15.60,20.69,0.74,51.31,62.29,59.80,0.00,0.00,0.00,8.57,20220530


# 2. 데이터 준비

In [ ]:
# 모델 평가
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_1 = df.copy()
df_1.columns

Index(['년', '월', '일', '요일', '회차', '나이', '성별', '신장', '체중', '체지방률', '이완기혈압',
       '수축기혈압', 'BMI', '유연성(cm)', '일리노이', '체공시간', '협응력', '심폐지구력', '근력', '학교폐쇄',
       '모임제한', '대중교통폐쇄', '시설제한', '측정일'],
      dtype='object')

In [ ]:
df_1['측정일'].head()
#train1
# train = df_1[df_1['측정일'] < 20220101]
# test = df_1[df_1['측정일'] >= 20220101]

#train2
# train = df_1[df_1['측정일'] < 20210101]
# test = df_1[df_1['측정일'] >= 20220101]

#train3
# train = df_1[df_1['측정일'] < 20200101]
# test = df_1[df_1['측정일'] >= 20220101]

#train4 - 코로나 전 2년 + 코로나 후 2년
# train = df_1[(df_1['측정일'] < 20220101) & (df_1['측정일'] > 20180101)]
# test = df_1[df_1['측정일'] >= 20220101]

#train5 - 코로나 시기만
train = df_1[(df_1['측정일'] < 20220101) & (df_1['측정일'] > 20200101)]
test = df_1[df_1['측정일'] >= 20220101]
# test.head()



In [ ]:
train_y = train[['체지방률']]
train_X = train.drop(['측정일', '체지방률', '년', '월', '일', '요일'], axis=1)
test_y = test[['체지방률']]
test_X = test.drop(['측정일', '체지방률', '년', '월', '일', '요일'], axis=1)
print(len(train_y), len(train_X))

50368 50368


In [ ]:
train_X

,회차,나이,성별,신장,체중,이완기혈압,수축기혈압,BMI,유연성(cm),일리노이,체공시간,협응력,심폐지구력,근력,학교폐쇄,모임제한,대중교통폐쇄,시설제한
209595,1.00,18.00,1.00,159.00,59.60,70.00,116.00,23.60,18.20,18.84,0.45,61.46,40.00,44.70,0.00,0.00,0.00,0.00
209596,1.00,18.00,0.00,172.50,60.80,83.00,123.00,20.40,11.40,17.71,0.57,46.81,21.00,59.70,0.00,0.00,0.00,0.00
209597,1.00,18.00,1.00,157.30,54.40,80.00,123.00,22.00,20.00,19.96,0.49,57.87,40.00,68.30,3.00,0.00,0.00,0.00
209598,1.00,18.00,1.00,156.10,48.50,60.00,111.00,19.90,4.60,20.65,0.48,36.72,32.00,46.80,3.00,0.00,0.00,0.00
209599,1.00,18.00,0.00,173.20,76.50,82.00,125.00,25.50,18.50,17.94,0.61,49.71,25.00,65.30,3.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259958,1.00,18.00,0.00,183.80,74.50,65.00,122.00,22.10,14.60,16.96,0.67,43.75,60.98,69.40,1.00,4.00,0.00,2.43
259959,1.00,13.00,0.00,171.90,103.20,80.00,114.00,34.90,12.30,23.15,0.40,63.02,8.00,31.00,1.00,4.00,0.00,2.43
259960,1.00,13.00,1.00,160.00,84.10,70.00,111.00,32.90,4.10,27.63,0.34,59.40,7.00,30.30,1.00,4.00,0.00,2.43
259961,1.00,13.00,0.00,167.60,90.60,79.00,133.00,32.30,2.00,24.59,0.39,66.81,16.00,28.10,1.00,4.00,0.00,2.43


### standard 스케일러

In [ ]:
train_X.columns

Index(['회차', '나이', '성별', '신장', '체중', '이완기혈압', '수축기혈압', 'BMI', '유연성(cm)',
       '일리노이', '체공시간', '협응력', '심폐지구력', '근력', '학교폐쇄', '모임제한', '대중교통폐쇄', '시설제한'],
      dtype='object')

# 여기서부터 TEST 시작

In [ ]:
col1 = ['나이','성별', '신장', '체중']
# col2 = ['지역_0', '지역_1', '지역_2', '지역_3', '지역_4','년생', '측정나이_0', '측정나이_1', '측정나이_2','측정회원성별_0', '측정회원성별_1', '신장', '체중', 
#         윗몸말아올리기, 반복점프, '협응력', 20m왕복]
col3 = ['지역_0', '지역_1', '지역_2', '지역_3', '지역_4','년생', '측정나이_0', '측정나이_1', '측정나이_2','측정회원성별_0', '측정회원성별_1', '신장', '체중']
# df.rename(columns={'왕복오래달리기(VO₂max)':'심폐지구력',
#                    '상대악력(%)':'근력',
#                    '반복점프':'근지구력',
#                    '앉아윗몸앞으로굽히기(cm)':'유연성',
#                    '일리노이':'민첩성',
#                    '체공시간':'순발력',
#                    '협응력시간':'협응력'}, inplace = True)
train_X = train_X[col1]
test_X = test_X[col1]

In [ ]:
print(train_X.shape, test_X.shape)

(50368, 4) (43150, 4)


In [ ]:
#PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures  # 표준 척도구성 장치

order = 2
drop_col = train_X.columns
poly = PolynomialFeatures(degree=order, include_bias=False)
poly.fit(train_X)

df_tt = poly.transform(train_X) # poly 변환 후 결과가 ndarray format임.
train_X = pd.merge(train_X, pd.DataFrame(df_tt), how='left', on=train_X.index)
train_X.drop(columns=drop_col)
train_X.drop(columns=train_X.columns[0], inplace=True)

df_ttt = poly.transform(test_X) # poly 변환 후 결과가 ndarray format임.
test_X = pd.merge(test_X, pd.DataFrame(df_ttt), how='left', on=test_X.index)
test_X.drop(columns=drop_col)
test_X.drop(columns=test_X.columns[0], inplace=True)

print(train_X.shape, test_X.shape)

(50368, 18) (43150, 18)


In [ ]:
test_X.tail()

,나이,성별,신장,체중,0,1,2,3,4,5,6,7,8,9,10,11,12,13
43145,18.00,0.00,177.70,70.90,18.00,0.00,177.70,70.90,324.00,0.00,3198.60,1276.20,0.00,0.00,0.00,31577.29,12598.93,5026.81
43146,18.00,1.00,165.50,60.80,18.00,1.00,165.50,60.80,324.00,18.00,2979.00,1094.40,1.00,165.50,60.80,27390.25,10062.40,3696.64
43147,16.00,1.00,167.60,61.50,16.00,1.00,167.60,61.50,256.00,16.00,2681.60,984.00,1.00,167.60,61.50,28089.76,10307.40,3782.25
43148,16.00,1.00,152.80,49.70,16.00,1.00,152.80,49.70,256.00,16.00,2444.80,795.20,1.00,152.80,49.70,23347.84,7594.16,2470.09
43149,15.00,0.00,158.60,50.20,15.00,0.00,158.60,50.20,225.00,0.00,2379.00,753.00,0.00,0.00,0.00,25153.96,7961.72,2520.04


In [ ]:
# standard스케일러
standard_scaler = StandardScaler()
train_X_standard_scaler = standard_scaler.fit_transform(train_X)
train_X_standard_scaler = pd.DataFrame(train_X_standard_scaler, columns=[col for col in train_X.columns])
test_X_standard_scaler = standard_scaler.transform(test_X)
test_X_standard_scaler = pd.DataFrame(test_X_standard_scaler, columns=[col for col in test_X.columns])
test_X_standard_scaler.head()

,나이,성별,신장,체중,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.20,-0.83,0.49,0.45,1.20,-0.83,0.49,0.45,1.25,-0.82,1.19,0.89,-0.83,-0.82,-0.80,0.47,0.47,0.33
1,1.20,-0.83,0.06,0.24,1.20,-0.83,0.06,0.24,1.25,-0.82,1.00,0.68,-0.83,-0.82,-0.80,0.03,0.19,0.12
2,0.01,-0.83,0.23,0.63,0.01,-0.83,0.23,0.63,-0.04,-0.82,0.09,0.52,-0.83,-0.82,-0.80,0.20,0.57,0.52
3,-0.58,-0.83,0.92,1.69,-0.58,-0.83,0.92,1.69,-0.63,-0.82,-0.15,1.08,-0.83,-0.82,-0.80,0.91,1.68,1.73
4,-0.58,-0.83,1.45,0.64,-0.58,-0.83,1.45,0.64,-0.63,-0.82,0.04,0.25,-0.83,-0.82,-0.80,1.48,0.85,0.53


# 3. 모델

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
train_X_standard_scaler.head()

,나이,성별,신장,체중,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.20,1.21,-1.02,-0.35,1.20,1.21,-1.02,-0.35,1.25,1.47,0.53,0.12,1.21,1.19,1.25,-1.01,-0.51,-0.41
1,1.20,-0.83,0.60,-0.27,1.20,-0.83,0.60,-0.27,1.25,-0.82,1.24,0.20,-0.83,-0.82,-0.80,0.58,-0.15,-0.34
2,1.20,1.21,-1.22,-0.71,1.20,1.21,-1.22,-0.71,1.25,1.47,0.44,-0.22,1.21,1.17,1.07,-1.21,-0.84,-0.70
3,1.20,1.21,-1.36,-1.12,1.20,1.21,-1.36,-1.12,1.25,1.47,0.38,-0.60,1.21,1.15,0.87,-1.34,-1.19,-1.00
4,1.20,-0.83,0.68,0.82,1.20,-0.83,0.68,0.82,1.25,-0.82,1.27,1.23,-0.83,-0.82,-0.80,0.66,0.84,0.72


In [ ]:
# train_X, valid_X, train_y, valid_y = train_test_split(df_X_standard_scaler, df_y, train_size=0.8, test_size = 0.2, random_state= 3766)

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

rg_model = Ridge(alpha=1, solver="cholesky")
rg_model.fit(train_X_standard_scaler, train_y)

Ridge(alpha=1, solver='cholesky')

In [ ]:
pred_train = rg_model.predict(train_X_standard_scaler)
pred_test = rg_model.predict(test_X_standard_scaler)


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler  # 표준 척도구성 장치

metrics = []

for y, pred_y in [(train_y, pred_train), (test_y, pred_test)]:
    MAE = mean_absolute_error(y, pred_y)
    MSE = mean_squared_error(y, pred_y)
    RMSE = np.sqrt(mean_squared_error(y, pred_y))
    R2 = r2_score(y, pred_y)
    metrics.append((MAE, MSE, RMSE, R2))

pd.DataFrame({'train':metrics[0], 'test':metrics[1]}, index=['MAE', 'MSE', 'RMSE', 'R2'])
# 평균제곱오차(0에 가까울수록 예측력이 좋음)
# r2 = 결정계수(1에 가까울수록 예측력이 좋음)

---
# 여기까지!!

In [ ]:
# 모델 평가
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import LassoCV , ElasticNetCV , RidgeCV
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, KFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor 
from sklearn.cross_decomposition import PLSRegression 
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

In [ ]:
# Ridge
pipe = Pipeline([('scl', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
param = [{"poly__degree" : np.arange(1,3), "model__alpha" : [0.01, 0.1, 1, 10], "model__solver": [ "cholesky", "lsqr", "sparse_cg"]}]
grid = GridSearchCV(pipe, param, cv=5, scoring  = 'neg_mean_squared_error', return_train_score=True)
grid.fit(train_X, train_y)
pred_y = grid.predict(test_X)

In [ ]:
print(f"최적의 하이퍼 파라미터 : {grid.best_params_}\n")
print(f"최적의 모델 평균 성능 : {grid.best_score_}\n")
print(f'MSE: {mean_squared_error(test_y, pred_y):.4f}')
print(f"R2: {r2_score(test_y, pred_y):.4f}\n")

최적의 하이퍼 파라미터 : {'model__alpha': 1, 'model__solver': 'cholesky', 'poly__degree': 2}

최적의 모델 평균 성능 : -12.978390016186129

MSE: 13.0895
R2: 0.8501



In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

model = 